# ABS Inflation multi-measure

## Python set-up

In [1]:
# system imports
from dataclasses import dataclass
from pathlib import Path
from typing import TypeAlias, TypeVar

# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# local imports
from abs_data_capture import (
    AbsLandingPage,
    find_id,
    get_abs_data,
    get_fs_constants,
    metacol,
)
from plotting import clear_chart_dir, finalise_plot, line_plot, set_chart_dir
from utility import qtly_to_monthly

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

### Typing information

In [2]:
@dataclass
class ABSSelectInput:
    """Data used to select m."""

    landing_page: AbsLandingPage
    table: str
    orig_sa: str
    search1: str
    search2: str
    abbr: str
    calc_growth: bool


@dataclass
class AbsSelectOutput:
    """For each series returned."""

    series: pd.Series
    cat_id: str
    table: str
    abbr: str


# Note: use NewType in the future when moved to Python 3.12
MeasuresType: TypeAlias = dict[str, ABSSelectInput]
AbsMultiSeries: TypeAlias = dict[str, AbsSelectOutput]

### Data capture

In [3]:
measuers: MeasuresType = {
    # specify the data items we wish to extract from the ABS ...
    "Qrtly All Groups CPI": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="2",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="All groups CPI ;  Australia ;",
        abbr="Q-CPI",
        calc_growth=False,
    ),
    "Qrtly CPI Trimmed Mean": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="consumer-price-index-australia",
        ),
        table="8",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Quarter of Previous Year",
        search2="Trimmed Mean ;  Australia ;",
        abbr="Q-CPI-TM",
        calc_growth=False,
    ),
    "Monthly CPI Indicator": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="All groups CPI, seasonally adjusted ;  Australia",
        abbr="M-CPI",
        calc_growth=False,
    ),
    "Monthly CPI Trimmed Mean": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="monthly-consumer-price-index-indicator",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage Change from Corresponding Month of Previous Year",
        search2="Annual Trimmed Mean ;  Australia ;",
        abbr="M-CPI-TM",
        calc_growth=False,
    ),
    "Producer Price Index": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="producer-price-indexes-australia",
        ),
        table="1",
        orig_sa="Orig",
        search1="Percentage change from corresponding quarter of previous year",
        search2="Final ;  Total ",
        abbr="PPI",
        calc_growth=False,
    ),
    "Wage Price Index": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="price-indexes-and-inflation",
            topic="wage-price-index-australia",
        ),
        table="1",
        orig_sa="SA",
        search1="Percentage Change From Corresponding Quarter of Previous Year",
        search2="Australia ;  Total hourly rates of pay excluding bonuses ;  "
        + "Private and Public ;  All industries ;",
        abbr="WPI",
        calc_growth=False,
    ),
    "Households final consumption Price Deflator": ABSSelectInput(
        landing_page=AbsLandingPage(
            theme="economy",
            parent_topic="national-accounts",
            topic="australian-national-accounts-national-income-expenditure-and-product",
        ),
        table="5",
        orig_sa="SA",
        search1=" ",
        search2="Households ;  Final consumption expenditure ;",
        abbr="HFCE",
        calc_growth=True,
    ),
    #    "Non-farm hourly employee compensation": ABSSelectInput(
    #        landing_page=AbsLandingPage(
    #            theme="economy",
    #            parent_topic="national-accounts",
    #            topic="australian-national-accounts-national-income-expenditure-and-product",
    #        ),
    #        table="24",
    #        orig_sa="SA",
    #        search1="Current prices ;",
    #        search2="Non-farm compensation of employees per hour:",
    #        abbr="NFHEC",
    #        calc_growth=True,
    #    ),
}

In [4]:
def get_multi_series(measures: MeasuresType) -> AbsMultiSeries:
    """Return a dictionary of Series data from the ABS,
    Once series for each item in the measure dictionary."""

    pool = {}
    for name, select in measures.items():
        print(name)

        # get the ABS data
        data_dict = get_abs_data(select.landing_page)
        _, _, Cat_ID, meta = get_fs_constants(data_dict, select.landing_page)
        data = data_dict[select.table]

        # get the specific series we want to plot
        search_terms = {
            select.table: metacol.table,
            {"SA": "Seasonally Adjusted", "Orig": "Original"}[
                select.orig_sa
            ]: metacol.stype,
            select.search1: metacol.did,
            select.search2: metacol.did,
        }
        series = data[find_id(meta, search_terms, verbose=True)[0]]
        index = series.index
        if select.calc_growth:
            periods = 4 if index.freqstr[0] == "Q" else 12
            series = (series / series.shift(periods) - 1) * 100.0
        pool[f"{name} ({select.orig_sa})"] = AbsSelectOutput(
            series=series,
            cat_id=Cat_ID,
            table=select.table,
            abbr=select.abbr,
        )

    return pool

In [5]:
dataset = get_multi_series(measuers)

Qrtly All Groups CPI
Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/009cab4e5b98e8a0836b649cecbac43a--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...
Qrtly CPI Trimmed Mean
Monthly CPI Indicator
Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/f32ae311e84e8db4484ecf18dcf6c27b--All-Time-Series-Spreadsheets.zip
Extracting DataFrames from the zip-file ...
Monthly CPI Trimmed Mean
Producer Price Index
Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/0b3053b171a4381f3e0724750f2ebf2d--Time-series-all.zip
Extracting DataFrames from the zip-file ...
Wage Price Index
Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/6052268df887aaf3a0774941b419dc9b--Time-series-spreadsheets-all.zip
Extracting DataFrames from the zip-file ...
Households final consumption Price Deflator
Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/5

## Plot the data

In [6]:
def plot_settings():
    """Plot settings."""

    chart_dir = "./CHARTS/Inflation"
    Path(chart_dir).mkdir(parents=True, exist_ok=True)
    clear_chart_dir(chart_dir)
    set_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


plot_settings()

In [7]:
TARGET = {
    "ymin": 2,
    "ymax": 3,
    "color": "#dddddd",
    "label": "2-3% inflation target",
    "zorder": -1,
}
LFOOTER = "Australia. Orig = Original series. " "SA = Seasonally adjusted series. "

In [8]:
def df_from_ams(mo: AbsMultiSeries) -> pd.DataFrame:
    """Get a dataframe from the ABS Multi Series item."""

    frame_dict = {key: val.series for key, val in mo.items()}
    for name, series in frame_dict.items():
        if series.index.freqstr[0] == "Q":
            series_m = qtly_to_monthly(series, interpolate=False)
            frame_dict[name] = series_m
    return pd.DataFrame(frame_dict)

In [9]:
def plot_multi_inflation(data: AbsMultiSeries):
    """Produce the mulit-mesure inflation charts."""

    frame = df_from_ams(data)
    source = "ABS " + ", ".join(
        [f"{x.cat_id.split(r'.', 1)[0]}-{x.table}" for x in data.values()]
    )
    latest = ", ".join(
        [
            f"{data[name].abbr} {frame[name].dropna().round(1).iloc[-1]}"
            for name in frame.columns
        ]
    )

    starts = ("1959-01-01", "2017-11-01")
    styles = (None, ["solid", "dotted", "dashed"] * 3)
    markers = (
        None,
        ["o", "v", "^", "<", ">", "8", "s", "p", "*", "h", "H", "D", "d", "P", "X"],
    )

    for start, style, marker in zip(starts, styles, markers):
        line_plot(
            frame,
            starts=start,
            style=style,
            marker=marker,
            markersize=6,
            dropna=True,
            title="Inflation measures",
            ylabel="Per Cent Annual Growth",
            legend={"fontsize": "xx-small"},
            axhspan=TARGET,
            y0=True,
            rfooter=source,
            lfooter=LFOOTER,
            rheader=latest if start != starts[0] else None,
            tags=start if start is not None else "",
            show=SHOW,
        )

In [10]:
plot_multi_inflation(dataset)

### Weighted CPI trajectory

In [11]:
def trajectory(data: AbsMultiSeries):

    frame = df_from_ams(data)

    THRESH = 2.5
    START_YEAR = 2019
    SERIES = "Qrtly CPI Trimmed Mean (SA)"
    cpiwm_mon = frame[SERIES][frame.index.year >= START_YEAR].dropna()
    cpiwm_q = cpiwm_mon[cpiwm_mon.index.month.isin([3, 6, 9, 12])]
    cpiwm_q.index = cpiwm_q.index.to_timestamp(how="end").to_period(freq="Q")
    start = cpiwm_q[cpiwm_q > 2.5].index[0]
    peak = cpiwm_q[cpiwm_q == cpiwm_q.max()].index[0]
    tail = cpiwm_q.index[-1]

    up_rate = (cpiwm_q[peak - 1] - cpiwm_q[start]) / ((peak - 1) - start).n
    down_rate = (cpiwm_q[tail] - cpiwm_q[peak + 1]) / (tail - (peak + 1)).n

    s = pd.Series()
    count = 0
    while True:
        count = count + 1
        pos = tail + count
        s[pos] = cpiwm_q[tail] + (down_rate * count)
        if s[pos] <= THRESH:
            break
    s = qtly_to_monthly(s, interpolate=False)

    ax = cpiwm_mon.plot(label=SERIES)
    s.plot(ax=ax, label="Current trajectory")

    for m in [
        "Monthly CPI Trimmed Mean (Orig)",
        # "Monthly CPI Indicator (SA)",
    ]:
        cpi_mon = frame[m][frame.index.year >= START_YEAR]
        cpi_mon.plot(ax=ax, lw=2, label=m)

    for x in s.index:
        ax.text(x, s[x], round(s[x], 1))
    ax.text(cpiwm_mon.index[-1], cpiwm_mon.iloc[-1], round(cpiwm_mon.iloc[-1], 1))

    finalise_plot(
        axes=ax,
        title=f"Current underlying inflation trajectory",
        ylabel="Per cent",
        legend={
            "fontsize": "xx-small",
            "loc": "upper left",
        },
        axhspan=TARGET,
        lfooter=LFOOTER,
        rfooter="Source: ABS 6401, 6484",
        show=SHOW,
    )

    print(up_rate, down_rate)

In [12]:
trajectory(dataset)

1.1666666666666665 -0.7666666666666666


### Plot Monthly v Quarterly pairs

In [13]:
def plot_mq(ams_data: AbsMultiSeries) -> None:
    """Plot some monthly v quarterly series."""

    frame = df_from_ams(ams_data)

    data_pairs = {
        # "pair name": ["a", "b"],
        "CPI": ("Monthly CPI Indicator (SA)", "Qrtly All Groups CPI (Orig)"),
        "Trimmed Mean": (
            "Monthly CPI Trimmed Mean (Orig)",
            "Qrtly CPI Trimmed Mean (SA)",
        ),
    }

    START_YEAR = 2019
    for name, (m, q) in data_pairs.items():
        data_m = frame.loc[frame.index.year >= START_YEAR, m]
        data_q = frame.loc[frame.index.year >= START_YEAR, q]
        data_q = data_q.interpolate(limit_area="inside", limit=2)
        ax = data_q.plot(label=q)
        data_m.plot(ax=ax, lw=1.5, label=m)

        finalise_plot(
            axes=ax,
            title=f"{name}: Monthly Vs Quarterly",
            ylabel="Per cent",
            legend={
                "fontsize": "xx-small",
                "loc": "upper left",
            },
            axhspan=TARGET,
            lfooter=LFOOTER,
            rfooter="Source: ABS 6401, 6484",
            y0=True,
            show=SHOW,
        )

In [14]:
plot_mq(dataset)

## Finished

In [15]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Mon Feb 19 2024 11:20:31

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

matplotlib: 3.8.3
numpy     : 1.26.4
pandas    : 2.2.0

Watermark: 2.4.3



In [16]:
print("Finished")

Finished
